<a href="https://colab.research.google.com/github/masies/CRA/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
! pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html
! pip install -q transformers==3.5.0 fast-trees
! git clone -q https://github.com/microsoft/CodeXGLUE.git

from os import walk
import os
import regex

from google.colab import auth
auth.authenticate_user()
project_id = 'helical-loop-303918'
bucket_name = 'code_review_automation'
!gcloud config set project {project_id}

fatal: destination path 'CodeXGLUE' already exists and is not an empty directory.
Updated property [core/project].


In [13]:
!mkdir ./code_code
!mkdir ./codeANDcomment_code
!mkdir ./marked_code
!mkdir ./code_comment
!mkdir ./mixture



# !gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/big_model/code_code/validation_eval ./code_code
!gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/validation_eval ./codeANDcomment_code
# !gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/big_model/code_comment/validation_eval ./code_comment
# !gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/validation_eval ./marked_code
# !gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/big_model/mixture/slanted/validation_eval ./mixture

mkdir: cannot create directory ‘./code_code’: File exists
mkdir: cannot create directory ‘./codeANDcomment_code’: File exists
mkdir: cannot create directory ‘./marked_code’: File exists
mkdir: cannot create directory ‘./code_comment’: File exists
mkdir: cannot create directory ‘./mixture’: File exists
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/validation_eval/codeANDcomment_code_270000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/validation_eval/codeANDcomment_code_210000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/validation_eval/codeANDcomment_code_240000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/validation_eval/codeANDcomment_code_220000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/validation_eval/codeANDco

In [14]:
def eval(val_folder, tasks):
  for task in tasks:
    PPs = []
    APPs = []
    print(task)
    _, _, filenames = next(walk(val_folder))
    prediction_paths = [os.path.join(val_folder, f) for f in filenames if f.startswith(task) and f.endswith("predictions")]
    target_path = [os.path.join(val_folder, f) for f in filenames if f.startswith(task) and f.endswith("targets")][0]

    with open(target_path) as f:
        targets = [item.strip() for item in f.readlines()]

    for path in sorted(prediction_paths):
        step = regex.findall( task + "_(\d+)_predictions", path)[0]
        with open(path) as f:
          predictions = [item.strip() for item in f.readlines()]
        
        perfect_predictions = 0
        almost_perfect_predictions = 0
        for pred, target in zip(predictions, targets):
          if ' '.join(pred.strip().split()) == ' '.join(target.strip().split()):
            perfect_predictions += 1
          elif ''.join(pred.strip().split()) == ''.join(target.strip().split()):
            almost_perfect_predictions += 1
            index = next((i for i, (char1, char2) in enumerate(zip(pred, target)) if char1 != char2), None)
        PPs.append(round(perfect_predictions/len(predictions)*100,2))
        APPs.append(round((perfect_predictions + almost_perfect_predictions)/len(predictions)*100,2))
        # print(step + " - PP: ", round(perfect_predictions/len(predictions)*100,2), " | APP: ", round((perfect_predictions + almost_perfect_predictions)/len(predictions)*100,2))
        # print()
    print(PPs)
    print(APPs)

# eval('mixture/validation_eval', ["code_code", "codeANDcomment_code", "code_comment"])
# eval('code_code/validation_eval', ["code_code"])
eval('codeANDcomment_code/validation_eval', ["codeANDcomment_code"])
# eval('code_comment/validation_eval', ["code_comment"])
# eval('marked_code/validation_eval', ["marked_code"])
      


codeANDcomment_code
[0.0, 10.41, 11.71, 11.61, 11.7, 11.85, 12.1, 12.09, 12.15, 12.33, 12.32, 12.1, 12.42, 12.36, 12.46, 12.33, 12.67, 12.71, 12.63, 12.82, 12.61, 12.81, 12.65, 12.79, 13.19, 12.87, 13.07, 13.17, 13.21, 13.39]
[0.0, 10.9, 12.32, 12.22, 12.37, 12.57, 12.82, 12.82, 12.78, 12.96, 12.99, 12.73, 13.05, 13.03, 13.18, 13.06, 13.41, 13.44, 13.29, 13.53, 13.29, 13.44, 13.38, 13.45, 13.85, 13.55, 13.7, 13.89, 13.89, 14.07]


In [ ]:
# def codeBleu(pred, target):
#   with open("pred.txt", "w") as f:
#     f.write(pred)
#   with open("tar.txt", "w") as f:
#     f.write(target)
#   try :
#     a = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs ./../../../../../pred.txt --hyp ./../../../../../tar.txt --lang java --params 0.25,0.25,0.25,0.25
#     codeBleu = float(a[1].replace("CodeBLEU score:  ",""))
#     print(codeBleu)
#   except Exception as e:
#     codeBleu = 0

In [2]:
!rm -rf ./test
!mkdir test
!gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code ./test

Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/events.out.tfevents.1619464162.e232c288671b...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/events.out.tfevents.1619363505.359a39f484bc...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/events.out.tfevents.1619419802.063ad3149cc8...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/checkpoint...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/model.ckpt-200000.data-00000-of-00002...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/graph.pbtxt...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/model.ckpt-210000.data-00000-of-00002...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/model.ckpt-200000.data-00001-of-00002...
Cop

In [3]:
checkpoints = !find ./test -name "model.ckpt-*.index" -print0 | xargs -0 ls
checkpoints = [x.replace("./test/codeANDcomment_code/model.ckpt-","").replace(".index","") for x in checkpoints]
checkpoints

['200000',
 '210000',
 '210100',
 '220000',
 '220200',
 '230000',
 '230300',
 '240000',
 '240400',
 '250000',
 '250500',
 '260000',
 '260600',
 '270000',
 '270700',
 '280000',
 '280800',
 '290000',
 '290900',
 '300000',
 '301000',
 '310000',
 '311100',
 '320000',
 '321200',
 '330000',
 '331300',
 '341300',
 '341400',
 '351300',
 '351500',
 '361300',
 '361600',
 '371300',
 '371700',
 '381300',
 '381800',
 '391300',
 '391900',
 '401300',
 '402000',
 '411300',
 '412100',
 '421300',
 '422200',
 '431300',
 '432300',
 '441300',
 '442400',
 '451300',
 '452500',
 '461300',
 '462600',
 '472600',
 '472700',
 '482600',
 '482800',
 '492600',
 '492900',
 '500000']